# 2. Loading the Transport Properties ANNs models

This notebook is part of the Supporting Information of the article *Simulation and data-driven modeling of the transport properties of the Mie fluid* by Gustavo Chaparro and Erich A. Müller. (Submitted to Journal of Physical Chemistry B).

This work aims to compute and model the self diffusivity ($D^*$), shear viscosity ($\eta^*$), and thermal conductivity ($\kappa^*$) of systems of particles interacting through the Mie potential:

$$ \frac{\mathcal{U}^{Mie}}{\epsilon} = \underbrace{\left[ \frac{\lambda_r}{\lambda_r- \lambda_a} \left( \frac{\lambda_r}{\lambda_a}\right)^{\frac{\lambda_a}{\lambda_r - \lambda_a}}   \right]}_{\mathcal{C}^{Mie}} \left[ \left(\frac{\sigma}{r}\right)^{\lambda_r} -  \left(\frac{\sigma}{r}\right)^{\lambda_a} \right] $$ 

Here, $\mathcal{U}^{Mie}$ is the interaction energy between two particles, $\epsilon$ is the interaction energy well depth, $\sigma$ is the length scale, loosely associated to the effective particle diameter, and $r$ is the center-to-center distance between two Mie particles. Finally, $\lambda_r$ and $\lambda_a$ are the repulsive and attractive exponents, respectively. This work uses reduced units in which the Boltzmann constant ($k_b$), potential well depth ($\epsilon$), the shape parameter ($\sigma$), and Mie particle mass ($M$) are set to unity.

This notebook shows how to load the trained ANN models to compute the transport properties of the Mie Fluid. This notebook relies on the following packages:
- Numpy (tested on version 1.24.2)
- jax (tested on version 0.4.4)
- flax (tested on version 0.6.6)
- nest_asyncio
- tp_modeling (Package containing the Transport Properties and Helmholtz models based on ANN using JAX and Flax)

In [1]:
import numpy as np

from tp_modeling import helper_get_alpha # helper function to get alpha_vdw value of a Mie Fluid
from tp_modeling import HelmholtzModel   # Helmholtz energy model for a Mie Fluid (FE-ANN EoS)
from tp_modeling import TransportModel_PVT # Transport model for the Mie Fluid (TP = ANN(alpha, rho, T))
from tp_modeling import TransportModel_PVT_Tinv # Transport model for the Mie Fluid (TP = ANN(alpha, rho, 1/T))
from tp_modeling import TransportModel_entropy # Transport model for the Mie Fluid  based on entropy scaling (TP = ANN(alpha, S^res))
# scaling functions for diffusivity, viscosity and thermal conductivity (needed for entropy scaling)
from tp_modeling import diffusivity_scaling, viscosity_scaling, thermal_conductivity_scaling 
from tp_modeling import TransportModelResidual_PVT_Tinv # Transport model for the Mie Fluid (TP = TP[1] + [ANN(alpha, rho, 1/T) -  ANN(alpha, rho=0., 1/T)])
from tp_modeling import density_diffusivity_mie6_dilute, viscosity_mie6_dilute, thermal_conductivity_mie6_dilute # dilute gas approximations for diffusivity, viscosity and thermal conductivity
from tp_modeling import linear_activation

# some JAX/FLax imports
from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
from flax.core import freeze
from flax import linen as nn
import nest_asyncio
nest_asyncio.apply()

# seeting up the precission to float64
PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

## Transport property model description 

For all the ANNs, the Mie fluid is described by the $\alpha_{vdw}$ parameter, obtained as follows.

$$\alpha_{vdw} =  \mathcal{C}^{Mie} \left[ \frac{1}{\lambda_a - 3} - \frac{1}{\lambda_r - 3}\right]$$


This work models a transport property ($\gamma^*$) using three possible approaches.

1. `TransportModel_PVT`, where a transport property is modeled as follows: $\gamma^* = \textnormal{ANN}(\alpha_{vdw}, \rho^*, T^*)$.
2. `TransportModel_PVT_Tinv`, where a transport property is modeled as follows: $\gamma^* = \textnormal{ANN}(\alpha_{vdw}, \rho^*, 1/T^*)$.
3. `TransportModel_entropy`, where a scaled transport property is modeled as follows: $\tilde{\gamma}^* = \textnormal{ANN}(\alpha_{vdw}, S^{*, res})$.
4. `TransportModelResidual_PVT_Tinv`, where a transport property is modeled as follows: $\gamma^* = \gamma^*_{[1]} + [\textnormal{ANN}(\alpha_{vdw}, \rho^*, 1/T^*) - \textnormal{ANN}(\alpha_{vdw}, \rho^*=0, 1/T^*)]$.

The first two approaches are a direct correlation of the transport property ($\gamma^*$) at a given density ($\rho^*$) and temperature ($T^*$). 
The third approach corresponds to entropy scaling, in which a reduced transport property is said to be an univariate function of the residual entropy ($S^{*, res}$). In this framework, the scaling of the transport property is given below.

$$\tilde{D}^* = \frac{(\rho^*)^{1/3} D^* }{\sqrt{T^*}}; \quad \tilde{\eta}^* = \frac{\eta^*}{(\rho^*)^{2/3} \sqrt{T^*}}; \quad \tilde{\kappa}^* = \frac{\kappa^*}{(\rho^*)^{2/3} \sqrt{T^*}} $$

When using entropy scaling, the residual entropy has to be obtained from an equation of state (EoS). In this work, we decided to use the FE-ANN EoS, which has been trained from molecular dynamics data of the Mie fluid in a vast range of density and temperature. For more information about the FE-ANN EoS [see this article](https://doi.org/10.1063/5.0146634).

$$ A^{*, res} = \textnormal{ANN}(\alpha_{vdw}, \rho^*, 1/T^*) - \textnormal{ANN}(\alpha_{vdw}, \rho^*=0, 1/T^*) $$

### Loading the trained models

This repository's folder `../models` includes all the trained ANN parameters. We recommend using the following models:
- `rhodiff-rho-Tinv`: $\rho^* D^* = \textnormal{ANN}(\alpha_{vdw}, \rho^*, 1/T^*)$
- `logvisc-rho-Tinv-penalty-supcrit`: $\ln \eta^* = \textnormal{ANN}(\alpha_{vdw}, \rho^*, 1/T^*)$ (Trained with the penalty function)
- `logtcond-rho-Tinv`: $\ln \kappa^* = \textnormal{ANN}(\alpha_{vdw}, \rho^*, 1/T^*)$
- `residual-rhodiff-Tinv`:  $\rho^* D^* = \rho^* D^*_{[1]} + \left[ \textnormal{ANN}(\alpha_{vdw}(\lambda_r, 6), \rho^*, 1/T^*) - \textnormal{ANN}(\alpha_{vdw}(\lambda_r, 6), \rho^*=0, 1/T^*) \right]$

The models are loaded using the `checkpoints` function from the `flax.training` package. This function requires the directory where the model is saved and the name of the model. The  `checkpoints` function outputs a dictionary with relevant information about the model, like the parameters, architecture, and seed used to initialize the model. This information is saved on the `state_restore` variable. Once this information is available, the model can be initialized. In Jax/Flax, the parameters of the ANNs (weights and biases) are detached from the model. For this reason, they have to be stored in an external variable. 

The self diffusivity model is loaded in the next cell as an example.

In [2]:
######################################
# LOADING THE SELF DIFFUSIVITY MODEL #
######################################

# Folder where the model are stored
save_dir = '../models/self-diffusivity'
# Name of the model
model_name = "rhodiff-rho-Tinv"
prefix_tp = f'{model_name}-params'
step = None
# Loading the model
state_restored = checkpoints.restore_checkpoint(save_dir, target=None, prefix=prefix_tp)
tp_activation = state_restored['activation']
tp_features = list(state_restored['features'].values())

# Infering the model type
if 'residual' in model_name:
    TPModel = TransportModelResidual_PVT_Tinv
elif 'Tinv' in model_name:
    TPModel = TransportModel_PVT_Tinv
elif 'entropy' in model_name:
    TPModel = TransportModel_entropy
else:
    TPModel = TransportModel_PVT

# Getting the output activation function
if tp_activation == 'linear':
    output_activation = linear_activation
elif tp_activation == 'softplus':
    output_activation = nn.softplus

# ANN model
diff_model = TPModel(features=tp_features, output_activation=output_activation)
# ANN parameters
diff_params = freeze({'params': state_restored['params']})

The loaded model can predict the transport property using the `tp_model.apply` method. This method accepts floats, lists, or arrays as inputs for the $\alpha_{vdw}$, $\rho^*$, $T^*$ (or $\lambda_r$, $\rho^*$, $T^*$ for the $\lambda_r$-6 residual models). However, they all have to have the same length. For this reason, we recommend providing the inputs as arrays.

In [3]:
lambda_r = np.asarray([12.])
lambda_a = np.asarray([6.])
alpha = helper_get_alpha(lambda_r, lambda_a)
rhoad =np.array([0.8])
Tad = np.array([3.])

if 'residual' in model_name:
    model_inputs = (lambda_r, rhoad, Tad)
else:
    model_inputs = (alpha, rhoad, Tad)

diff_model.apply(diff_params, alpha, rhoad, Tad) # remember this models gives you rho*D

2024-01-17 16:39:47.049942: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none


Array([0.18607218], dtype=float64)

Even though we noticed that entropy scaling does not accurately describe the transport properties over the whole density range, this modeling approach can still be used. In this case, the transport property model and the FE-ANN EoS must be loaded. Both models are loaded following the same procedure as described above.

In [4]:
####################
# LOADING FE-ANN EOS
####################

save_dir = '../models/feanneos'
prefix_feann = 'params'
step = None
state_restored = checkpoints.restore_checkpoint(save_dir, target=None, prefix=prefix_feann)
helmholtz_features = list(state_restored['features'].values())
helmholtz_params = freeze({'params': state_restored['params']})
helmholtz_model = HelmholtzModel(features=helmholtz_features)

#########################
# LOADING TRANSPORT MODEL
#########################

save_dir = '../models/shear-viscosity'
model_name = "visc-entropy"
prefix_tp = f'{model_name}-params'
step = None
state_restored = checkpoints.restore_checkpoint(save_dir, target=None, prefix=prefix_tp)
tp_activation = state_restored['activation']
tp_features = list(state_restored['features'].values())

# Infering the model type
if 'residual' in model_name:
    TPModel = TransportModelResidual_PVT_Tinv
elif 'Tinv' in model_name:
    TPModel = TransportModel_PVT_Tinv
elif 'entropy' in model_name:
    TPModel = TransportModel_entropy
else:
    TPModel = TransportModel_PVT

if tp_activation == 'linear':
    output_activation = linear_activation
elif tp_activation == 'softplus':
    output_activation = nn.softplus

visc_model = TPModel(features=tp_features, output_activation=output_activation)
visc_params = freeze({'params': state_restored['params']})

In this case, the predictions of the transport property consist of a 3-step process.
1. The residual entropy is obtained from the FE-ANN EoS
2. The ANN is used to predict the scaled transport property
3. The transport property has to be unscaled. We provide the `diffusivity_scaling`, `viscosity_scaling` and `thermal_conductivity_scaling ` for this purpose.

In [5]:
lambda_r = 12.
lambda_a = 6.
alpha =np.array([helper_get_alpha(lambda_r, lambda_a)])
rhoad =np.array([0.8])
Tad = np.array([3.])

# First the residual entropy is obtained from the FE-ANN EOS
S_res = helmholtz_model.entropy_residual(helmholtz_params, alpha, rhoad, Tad)
# Then the transport properties are obtained from the entropy scaling model
scaled_visc = visc_model.apply(visc_params, alpha, S_res)
# Finally the transport properties are rescaled to the correct units
visc = viscosity_scaling(rhoad, Tad, scaled_visc, unscale=True)
visc

Array([1.82613783], dtype=float64)

In this work we also provide transport property models that separate the transport property between a dilute gas and a residual contribution. These models are only valid fro $\lambda_r-6$ Mie fluids. The final transport property is computed as follows:

$$ \rho^* D^* = \rho^* D^*_{[1]} + \left[ \textnormal{ANN}(\alpha_{vdw}(\lambda_r, 6), \rho^*, 1/T^*) - \textnormal{ANN}(\alpha_{vdw}(\lambda_r, 6), \rho^*=0, 1/T^*) \right]  $$ 
$$ \ln \eta^* = \ln  \eta^*_{[1]} + \left[ \textnormal{ANN}(\alpha_{vdw}(\lambda_r, 6), \rho^*, 1/T^*) - \textnormal{ANN}(\alpha_{vdw}(\lambda_r, 6), \rho^*=0, 1/T^*) \right] $$
$$ \ln \kappa^* = \ln  \kappa^*_{[1]} + \left[ \textnormal{ANN}(\alpha_{vdw}(\lambda_r, 6), \rho^*, 1/T^*) - \textnormal{ANN}(\alpha_{vdw}(\lambda_r, 6), \rho^*=0, 1/T^*) \right]$$

Here, $\rho^* D^*_{[1]}$, $\eta^*_{[1]}$, $\kappa^*_{[1]}$ are the dilute gas transport property obtained from kinetic theory. These are a function of a collision integral $\Omega^{(k,k)} and the temperature$ and are obtained as shown below.

$$ \rho^* D^*_{[1]} = \frac{3}{8} \frac{1}{\Omega^{(1,1)*}} \sqrt{\frac{T^*}{\pi}} $$
$$ \eta^*_{[1]} = \frac{5}{12} \frac{1}{\Omega^{(2,2)*}}\sqrt{\frac{T^*}{\pi}} $$ 
$$ \kappa^*_{[1]} = \frac{25}{32} \frac{C_V^*}{\Omega^{(2,2)*}}\sqrt{\frac{T^*}{\pi}} $$ 

This models are loaded similarly as the one shown above. Here is an example for the 

In [6]:
###################################################
# LOADING THE RESIDUAL THERMAL CONDUCTIVITY MODEL #
###################################################

# Folder where the model are stored
save_dir = '../models/thermal-conductivity'
# Name of the model
model_name = "residual-logtcond-rho-Tinv"
prefix_tp = f'{model_name}-params'
step = None
# Loading the model
state_restored = checkpoints.restore_checkpoint(save_dir, target=None, prefix=prefix_tp)
tp_activation = state_restored['activation']
tp_features = list(state_restored['features'].values())

# Infering the model type
if 'residual' in model_name:
    TPModel = TransportModelResidual_PVT_Tinv
elif 'Tinv' in model_name:
    TPModel = TransportModel_PVT_Tinv
elif 'entropy' in model_name:
    TPModel = TransportModel_entropy
else:
    TPModel = TransportModel_PVT

# Getting the output activation function
if tp_activation == 'linear':
    output_activation = linear_activation
elif tp_activation == 'softplus':
    output_activation = nn.softplus

# ANN model
residual_tcond_model = TPModel(features=tp_features, output_activation=output_activation)
# ANN parameters
residual_tcond_params = freeze({'params': state_restored['params']})

The main difference is that these residual model use the $\lambda_r$ as input instead of $\alpha_{vdw}$. The $\alpha_{vdw}$ is computed internally for the $\lambda_r-6$ pair. 

Additionally, the dilute gas property can be computed with the `density_diffusivity_mie6_dilute`, `viscosity_mie6_dilute` or `thermal_conductivity_mie6_dilute` functions. These use $\lambda_r$ and $T^*$ as inputs.

In [7]:
lambda_r = 12.
lambda_a = 6.
alpha = np.atleast_1d(helper_get_alpha(lambda_r, lambda_a))
rhoad =np.array([0.8])
Tad = np.array([3.])

if 'residual' in model_name:
    model_inputs = (np.atleast_1d(lambda_r), rhoad, Tad)
else:
    model_inputs = (alpha, rhoad, Tad)

# residual contribution
lntcond_res = residual_tcond_model.apply(residual_tcond_params, *model_inputs) 
# dilute gas contribution
lntcond_dilute = np.log(thermal_conductivity_mie6_dilute(lambda_r, Tad))
# total thermal conductivity
lntcond = lntcond_dilute + lntcond_res
tcond = np.exp(lntcond)
tcond

array([7.70180562])